# Part 4: Disaster Strikes

In [11]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000/ -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 11464846221 (10.68 GB)
DFS Remaining: 11193565184 (10.42 GB)
DFS Used: 271281037 (258.71 MB)
DFS Used%: 2.37%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 77
	Missing blocks (with replication factor 1): 77
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.4:9866 (project-4-twhero-dn-2.project-4-twhero_default)
Hostname: abe275c95fbb
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 271281037 (258.71 MB)
Non DFS Used: 14339429491 (13.35 GB)
DFS Remaining: 11193565184 (10.42 GB)
DFS Used%: 1.05%
DFS Rem

In [ ]:
import time
time.sleep(30)

In [12]:
#q9
import requests
import math
leftblock = []
distrib = {}
offset = 0
length = 1048576
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
myrange = math.ceil(r.json()['FileStatus']['length'] / r.json()['FileStatus']['blockSize'])
for i in range(myrange):
    r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&length={length}&noredirect=true")
    if r.status_code == 403:
        distrib['lost'] = distrib.get('lost',0) + 1
    else:
        id = r.json()['Location'].split('/')[2]
        distrib[id] = distrib.get(id,0) + 1
        leftblock.append(offset)
    offset += length
distrib

{'lost': 77, 'abe275c95fbb:9864': 90}

In [13]:
#q10
import pyarrow as pa
import pyarrow.fs
mycount = 0
hdfs = pa.fs.HadoopFileSystem('boss', 9000)
with hdfs.open_input_file('/single.csv') as f:
    for b in leftblock:
        blockdata = f.read_at(1048576,b).decode('utf-8')
        mycount += blockdata.count('Single Family')
mycount

2023-10-23 00:36:45,107 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


240072